In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform

## Identify Niftis to Use as Maps
### Organize Maps for Utilization in a Folder

In [ ]:
## Paths Input Here
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/barotano_2022_networks/ADNI1_ADAS11_vox_tstat_fwep.nii'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/patient_data/AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    x_roi_names = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    out_dir = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/seeds/adni_maps'
    #out_dir = r'path to out dir here'
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\stats'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

analysis = "adni2_adascog11_95_threshold_fwe"

In [ ]:
def z_score_matrix(matrix):
    from scipy.stats import zscore
    print('--------------------------------Performing z_score--------------------------------')
    print('pre z score max: ', np.max(matrix), np.shape(matrix))
    z_matrix = zscore(matrix)
    z_matrix = np.nan_to_num(np.nan_to_num(z_matrix, nan=0, posinf=0, neginf=0))
    print('post z score max: ', np.max(z_matrix), np.shape(z_matrix))
    return z_matrix

In [ ]:
def fisher_z_transform(matrix):
    ##Perform fisher z transformation on r matrix. 
    rfz_matrix = np.arctanh(matrix)
    return rfz_matrix

In [ ]:
def threshold_matrix(matrix, threshold=0.5, probability=True, direction='bidirectional', binarize=True):
    import scipy.stats as st
    print('--------------------------------Performing threshold--------------------------------')
    high_threshold = threshold
    if probability == True:
        high_threshold = st.norm.ppf(threshold) #this generates the z score that indicates everything BELOW the threshold value
        low_threshold = st.norm.ppf(1-threshold) #this generates th
        print('z score high threshold is: ', high_threshold)
        print('z score low threshold is: ', low_threshold)
    if direction == 'keep_greater':
        print('pre threshold: ', np.count_nonzero(matrix))
        print('max z score in matrix is: ', np.max(matrix))
        print('min z score in matrix is: ', np.min(matrix))
        empty = np.ones(matrix.shape)
        empty[matrix < high_threshold] = 0
        matrix = matrix * empty
        print('post threshold: ', np.count_nonzero(matrix))
        print('max z score in matrix is: ', np.max(matrix))
        print('min z score in matrix is: ', np.min(matrix))
        print('I zero everything below threshold')
    elif direction == 'keep_lesser':
        matrix[matrix > low_threshold] = 0
        print('I will zero everything above threshold')
    elif direction == 'bidirectional':
        print('I will keep everything above and below the threshold')
        binary_mx = np.zeros(matrix.shape); binary_mx[:,:,:]=0
        binary_mx[matrix > high_threshold] = 1; vox_over = np.count_nonzero(binary_mx)
        print('voxels over threshold: ', vox_over)
        binary_mx[matrix < low_threshold] = -1; vox_under = np.count_nonzero(binary_mx)-vox_over; 
        print('voxels under and over threshold: ', vox_under)
        matrix = binary_mx
        print('max z score in matrix is: ', np.max(matrix))
        print('min z score in matrix is: ', np.min(matrix))
    else:
        print('Error encountered, direction not provided')
        
    if binarize:
        matrix[matrix != 0] = 1
    return matrix

In [ ]:
def overlap(mask_matrix=None, target_matrix=r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\niftis_to_overlap\Fx-DBS-Network_N46.nii', 
            mask=False, dozscore=False, do_threshold=False, do_rfz=False, threshold=0.95, give_og=False, direction='keep_greater'):
    #import the matrix and prepare it.
    img = image.load_img(target_matrix); mx = img.get_fdata(); mx = np.nan_to_num(mx, nan=0, posinf=10, neginf=-10)
    if do_rfz:
        mx = fisher_z_transform(mx) 
    if dozscore:
        mx = z_score_matrix(mx)
    if do_threshold:
        mx = threshold_matrix(mx, threshold=threshold, direction=direction, probability=dozscore)
        og_mx=mx
    if dozscore & do_threshold == False: #Just normalize the matrix instead.
        overlap_mx = []
        if overlap_mx == []:
            overlap_mx = np.zeros(mx.shape) #generate empty matric
            overlap_mx = overlap_mx + mx/np.max(abs(mx)) #Fill empty matrix with normalized values from the path matrix
    else:
        overlap_mx = mx

    #Perform overlapping process.     
    if mask == True:
        for path in mask_matrix:
            img = image.load_img(path); mx = img.get_fdata(); mx = np.nan_to_num(mx, nan=0, posinf=0, neginf=0) #Import the target mask image from path in glob
            overlap_mx[mx==0] = 0
            print('Max value in mask: ', np.max(overlap_mx))

    if dozscore & do_threshold == False:
        return overlap_mx/np.max(abs(overlap_mx))
    elif give_og==True:
        print('OG Returned')
        return og_mx
    else:
        return overlap_mx

## Create Maps

In [ ]:
threshold = 0.95
threshold_str = str(threshold).split('.')
target_matrix = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/barotano_2022_networks/ADNI2_ADAS11_vox_tstat_fewp.nii'
##----------------------------------------------------------------User Input Above----------------------------------------------------------------
thresh_mx  = overlap(target_matrix=target_matrix, dozscore=False, do_rfz=False, do_threshold=True, threshold=threshold, mask=False, give_og=True, direction="keep_greater")
mask = nimds.get_img("mni_icbm152")
ovr_img1 = image.new_img_like(mask, thresh_mx)
ovr_html1 = plotting.view_img(ovr_img1, cut_coords=(0,0,0), title=(f'unmasked&thresh zscore: {threshold}'), black_bg=False, opacity=.75, cmap='ocean_hot')

In [ ]:
ovr_html1

In [ ]:
save = True

#----------------------------------------------------------------Save if desired----------------------------------------------------------------
analysis_name = analysis + threshold_str[1]
if save:
    if os.path.isdir(out_dir)==False:
        os.mkdir(out_dir)
    savename = analysis_name + '_negative_values_raw_map'
    #Save
    ovr_img1.to_filename(os.path.join(out_dir, f'{savename}'))
    ovr_html1.save_as_html(os.path.join(out_dir, f'{savename}.html'))

    print('Title: ' + savename)
    print('saved to: ', out_dir)

In [ ]:
## Use the above matrix to define the seeds to isolate by their coordinations
## Prepare to slice matrix to isolate seeds
roi_location = 'negative_r_right_anteroiferior_cerebellar_hemisphere'

sagittal_slice_left = -22 #----------------------------------------------------------------x vals
sagittal_slice_right = 44 #----------------------------------------------------------------x vals

coronal_slice_posterior = -44 #----------------------------------------------------------------y vals
coronal_slice_anterior = 90 #----------------------------------------------------------------y vals

axial_slice_inferior = -46 #----------------------------------------------------------------z vals
axial_slice_superior = 94 #----------------------------------------------------------------z vals

#----------------------------------------------------------------User Input Required----------------------------------------------------------------
x_min = -100; x_max = 100; xdelta = abs(x_min)+x_max
y_min = -132; y_max = 100; ydelta = abs(y_min)+y_max
z_min = -70; z_max = 120; zdelta = abs(z_min)+z_max

#this finds the proportionate location of the coordinates
xbound1 = (abs(x_min)+sagittal_slice_left)/xdelta; xbound2 = (abs(x_min)+sagittal_slice_right)/xdelta 
ybound1 = (abs(y_min)+coronal_slice_posterior)/ydelta; ybound2 = (abs(y_min)+coronal_slice_anterior)/ydelta
zbound1 = (abs(z_min)+axial_slice_inferior)/zdelta; zbound2 = (abs(z_min)+axial_slice_superior)/zdelta

In [ ]:
def slice_seed_out(thresh_mx, xbound1, xbound2, ybound1, ybound2, zbound1, zbound2, test=False):
    #Prepare matrices for masking
    blank_mx = np.zeros(thresh_mx.shape)
    xmask = blank_mx; ymask = blank_mx; zmask = blank_mx

    #Set direction to slice in by indexing length of the matrix
    x_len = len(thresh_mx[:,1,1]); y_len = len(thresh_mx[1,:,1]); z_len = len(thresh_mx[:,:,1])

    #Set the proportional location of the indices to slice
    xind1 = int(x_len*xbound1); xind2 = int(x_len*xbound2)
    yind1 = int(y_len*ybound1); yind2 = int(y_len*ybound2)
    zind1 = int(z_len*zbound1); zind2 = int(z_len*zbound2)
    print(xind1, xind2)
    print(x_len-xind1, x_len-xind2)
    # #set the mask to 'one' what occurs within the bounds of the indices
    # xmask[xind1:xind2,:,:] = 1; 
    # ymask[:,:,:] = 1
    # zmask[:,:,:] = 1
    # print(blank_mx.shape)
    # # zmask[:,:,zind1:zind2] = 1
    # # ymask[:,yind1:yind2,:] = 1
    # # zmask[:,:,zind1:zind2] = 1

    # mask_mx = xmask*ymask*zmask
    if test==True:
        blank_mx[:,:,:] = 0
        # blank_mx[x_len-xind1:x_len-xind2,10:60,40:90] = 1
        blank_mx[x_len-xind2:x_len-xind1,yind1:yind2,zind1:zind2] = 1
        masked_thresh = blank_mx
        print('Testing engaged')
    else:
        #create a boolean mask
        blank_mx[:,:,:] = 0
        blank_mx[x_len-xind2:x_len-xind1,yind1:yind2,zind1:zind2] = 1
        print(np.count_nonzero(blank_mx), blank_mx.shape, type(blank_mx))
        print(np.count_nonzero(thresh_mx), thresh_mx.shape, type(thresh_mx))

        #mask the matrix
        blank_mx[thresh_mx == 0] = 0
        masked_thresh = blank_mx
        print(np.count_nonzero(masked_thresh), masked_thresh.shape, type(masked_thresh))
    return masked_thresh

In [ ]:
# masked_thresh_mx = slice_seed_out(thresh_mx, xbound1=xbound1, xbound2=xbound2, ybound1=ybound1, ybound2=ybound2, zbound1=zbound1, zbound2=zbound2, test=False)
# ovr_img2 = image.new_img_like(mask, masked_thresh_mx);
# ovr_html2 = plotting.view_img(ovr_img2, cut_coords=(0,0,0), title=(f'thresh prob: {threshold}'), black_bg=False, opacity=.75, cmap='ocean_hot');
# ovr_html2

In [ ]:
#Generate Title of Save File
try:   
    threshnum = str(threshold).split('.')[1]
    savename = analysis + f'_thresholded_to_{str(threshnum)}' + f'{roi_location}'
except:
    # maskname = os.path.basename(globbed[0]).split('.nii')[0]
    # analysis2 = analysis + '_masked_by_' + maskname
    # print('Title: ' + analysis2)
    print('error')

In [ ]:
# if os.path.isdir(out_dir)==False:
#     os.mkdir(out_dir)

# #Save
# threshold = str(threshold).split(".")[1]
# ovr_img2.to_filename(os.path.join(out_dir, f'{savename}'))
# ovr_html2.save_as_html(os.path.join(out_dir, f'{savename}.html'))

# print('Title: ' + savename)
# print('saved to: ', out_dir)

## Generate Spherical ROI Around Coordinates

In [ ]:
def gen_sphere_roi(thresh_mx, xcoord, ycoord, zcoord, mask=False, radius=10):
    from nltools.mask import create_sphere
    #Prepare matrices for masking
    sphere_mask = create_sphere([xcoord, ycoord, zcoord], radius=radius)
    sphere_mask = sphere_mask.get_fdata()
    print(np.count_nonzero(sphere_mask))
    if mask:
        print(np.count_nonzero(thresh_mx))
        thresh_mx[sphere_mask == 0] = 0# != sphere_mask] = 0
        print(np.count_nonzero(thresh_mx))
        out_mx = thresh_mx
        print(np.count_nonzero(out_mx))
        print('I will mask to the regino around the sphere')
    else:
        out_mx = sphere_mask
        print('I will binarize wihtin the sphere')
    return out_mx

In [ ]:
x = -6
y = -41
z = 3

pre_mx = nimds.get_img("mni_icbm152")
mask = nimds.get_img("mni_icbm152")

sphere_roi = gen_sphere_roi(pre_mx, xcoord=x, ycoord=y, zcoord=z, mask=False, radius=10);
ovr_img3 = image.new_img_like(mask, sphere_roi);
ovr_html3 = plotting.view_img(ovr_img3, cut_coords=(0,0,0), black_bg=False, opacity=.75, cmap='ocean_hot');
ovr_html3

In [ ]:
#Generate Title of Save File
savename = analysis + f'_{threshold_str[1]}_{roi_location}'
print(savename)

In [ ]:
if os.path.isdir(out_dir)==False:
    os.mkdir(out_dir)

#Save
ovr_img3.to_filename(os.path.join(out_dir, f'{savename}'))
ovr_html3.save_as_html(os.path.join(out_dir, f'{savename}.html'))

print('Title: ' + savename)
print('saved to: ', out_dir)